In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[9],7
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[7]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.002339                    1  0.000025
0.000750        2 -0.002328                    2  0.000013
0.001052        3 -0.002302                    3  0.000077
0.001476        4 -0.002275                    4  0.000104
0.002070        5 -0.002245                    5  0.000135
0.002904        6 -0.002213                    6  0.000163
0.004074        7 -0.002179                    7  0.000196
0.005714        8 -0.002144                    8  0.000229
0.008015        9 -0.002110                    9  0.000262
0.011243       10 -0.002074                   10  0.000297
0.015771       11 -0.002039                   11  0.000333
0.022122       12 -0.002006                   12  0.000371
0.031031       13 -0.001976                   13  0.000409
0.043528       14 -0.001938                   14  0.000456
0.061057       15 -0.001706                   15  0.000659
0.085645       16 -0.001022                   16  0.001170
0.120136       17  0.000194                   17  0.001994
0.168516       18  0.001998                   18  0.003141
0.236378       19  0.004568                   19  0.004708
0.331549       20  0.008137                   20  0.006827
0.465100       21  0.013001                   21  0.009676
0.652400       22  0.018787                   22  0.013076
0.915100       23  0.020538                   23  0.014440
1.283650       24  0.017436                   24  0.013196
1.800600       25  0.013301                   25  0.011153
2.525700       26  0.009091                   26  0.008762
3.542800       27  0.005938                   27  0.006687
4.969550       28  0.003831                   28  0.005027
6.970850       29  0.002460                   29  0.003708
9.778100       30  0.001578                   30  0.002659
13.715850      31  0.001004                   31  0.001831
19.239350      32  0.000613                   32  0.001175
26.987250      33  0.000326                   33  0.000653
37.855300      34  0.000088                   34  0.000225
53.100050      35 -0.000230                   35 -0.000206
73.887500      36 -0.000533                   36 -0.000564
97.662500      37 -0.000686                   37 -0.000737
121.437500     38 -0.000695                   38 -0.000751
145.212500     39 -0.000560                   39 -0.000624
168.987500     40 -0.000357                   40 -0.000423
192.762500     41 -0.000011                   41 -0.000080
216.537500     42  0.000976                   42  0.000917
240.312500     43  0.003050                   43  0.002720
264.087500     44  0.006380                   44  0.005466
287.862500     45  0.011093                   45  0.010678
311.637500     46  0.017297                   46  0.017315
335.412500     47  0.024992                   47  0.023557
359.187500     48  0.033631                   48  0.031024
382.962500     49  0.041574                   49  0.039716
406.737500     50  0.051361                   50  0.049954
430.512500     51  0.062585                   51  0.059958
454.287500     52  0.074948                   52  0.070142
478.062500     53  0.088141                   53  0.082851
501.837500     54  0.101695                   54  0.097646
525.612500     55  0.115445                   55  0.114625
549.387500     56  0.128890                   56  0.129732
573.162500     57  0.141863                   57  0.142755
596.937500     58  0.154412                   58  0.153022
620.712500     59  0.166588                   59  0.162446
644.487500     60  0.179091                   60  0.173521
668.262500     61  0.192112                   61  0.187436
692.037500     62  0.203990                   62  0.202411
715.812500     63  0.210877                   63  0.213002
739.587500     64  0.220012                   64  0.227348
763.362500     65  0.212934                   65  0.223451
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -16.985901  0.000000e+00 -16.985901 -17.305941   
0.000624    2     -16.985901  6.945626e-08 -16.985901 -17.305941   
0.000876    3     -16.985902  9.792714e-08 -16.985901 -17.305941   
0.001229    4     -16.985902  1.393033e-07 -16.985902 -17.305941   
0.001723    5     -16.985902  1.994802e-07 -16.985902 -17.305941   
0.002417    6     -16.985902  2.869742e-07 -16.985902 -17.305941   
0.003391    7     -16.985902  4.141948e-07 -16.985902 -17.305941   
0.004757    8     -16.985903  5.991586e-07 -16.985902 -17.305941   
0.006672    9     -16.985904  8.679898e-07 -16.985903 -17.305941   
0.009359    10    -16.985905  1.258181e-06 -16.985904 -17.305941   
0.013128    11    -16.985906  1.824567e-06 -16.985905 -17.305942   
0.018415    12    -16.985908  2.645982e-06 -16.985906 -17.305942   
0.025830    13    -16.985911  3.835777e-06 -16.985908 -17.305942   
0.036232    14    -16.985916  5.557410e-06 -16.985910 -17.305943   
0.050823    15    -16.985921  8.068197e-06 -16.985913 -17.305943   
0.071291    16    -16.985930  1.216022e-05 -16.985917 -17.305944   
0.100000    17    -16.985941  1.988259e-05 -16.985921 -17.305944   
0.140271    18    -16.985955  3.505121e-05 -16.985920 -17.305943   
0.196760    19    -16.985971  6.448610e-05 -16.985907 -17.305937   
0.275997    20    -16.985985  1.207960e-04 -16.985864 -17.305924   
0.387100    21    -16.985983  2.268441e-04 -16.985757 -17.305891   
0.543100    22    -16.985940  4.234737e-04 -16.985517 -17.305821   
0.761700    23    -16.985800  7.697034e-04 -16.985030 -17.305681   
1.068500    24    -16.985545  1.261390e-03 -16.984284 -17.305461   
1.498800    25    -16.985216  1.820572e-03 -16.983395 -17.305182   
2.102400    26    -16.984835  2.391133e-03 -16.982444 -17.304857   
2.949000    27    -16.984443  2.910659e-03 -16.981532 -17.304514   
4.136600    28    -16.984056  3.358779e-03 -16.980697 -17.304166   
5.802500    29    -16.983680  3.739039e-03 -16.979941 -17.303827   
8.139200    30    -16.983324  4.064373e-03 -16.979260 -17.303514   
11.417000   31    -16.983002  4.355382e-03 -16.978647 -17.303254   
16.014700   32    -16.982736  4.636648e-03 -16.978100 -17.303084   
22.464000   33    -16.982564  4.932747e-03 -16.977632 -17.303060   
31.510500   34    -16.982547  5.264352e-03 -16.977282 -17.303263   
44.200100   35    -16.982786  5.636107e-03 -16.977150 -17.303825   
62.000000   36    -16.983570  5.935569e-03 -16.977634 -17.305011   
85.775000   37    -16.985212  6.076599e-03 -16.979136 -17.307046   
109.550000  38    -16.987273  6.204379e-03 -16.981068 -17.309351   
133.325000  39    -16.989672  6.645990e-03 -16.983026 -17.311875   
157.100000  40    -16.992364  7.760304e-03 -16.984604 -17.314600   
180.875000  41    -16.995481  9.871070e-03 -16.985610 -17.317667   
204.650000  42    -16.999992  1.434941e-02 -16.985642 -17.322038   
228.425000  43    -17.009603  2.670859e-02 -16.982894 -17.331357   
252.200000  44    -17.027356  5.305047e-02 -16.974305 -17.348534   
275.975000  45    -17.056861  1.005240e-01 -16.956337 -17.376962   
299.750000  46    -17.102102  1.770051e-01 -16.925097 -17.420302   
323.525000  47    -17.167252  2.908691e-01 -16.876383 -17.482469   
347.300000  48    -17.256618  4.506199e-01 -16.805998 -17.567899   
371.075000  49    -17.373405  6.621233e-01 -16.711282 -17.679625   
394.850000  50    -17.513676  9.194803e-01 -16.594196 -17.813772   
418.625000  51    -17.678701  1.229153e+00 -16.449547 -17.971500   
442.400000  52    -17.869361  1.596075e+00 -16.273287 -18.153676   
466.175000  53    -18.086167  2.023958e+00 -16.062209 -18.361197   
489.950000  54    -18.329334  2.515359e+00 -15.813975 -18.593735   
513.725000  55    -18.598361  3.070790e+00 -15.527570 -18.851600   
537.500000  56    -18.892742  3.690301e+00 -15.202441 -19.133679   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')